In [1]:
import numpy as np
import random
import os
import time
import torch
from torch.autograd import Variable
from copy import deepcopy

In [2]:
from lib.turn_program_into_file import turn_program_into_file
from game.wrappers.game_container import GameContainer

In [3]:
cpus = 4
player_size = 64
game = GameContainer(512, 512)
# Each run taks 2.6 sec per cpu, 38089 = a 24h run.
run_limit = 10
save_cycle = 2
output_dir = "./output/pyramid/"
run = int(time.time())
save_dir = output_dir + str(run) + '/'

In [4]:
# king_hill_ids = list(range(0, cpus))
# os.makedirs(os.path.join(output_dir, str(run)))

In [5]:
def return_result(game_result):
    ones = []
    twos = []
    
    for i in game_result:
        if i == 1:
            ones.append(1)
            twos.append(0)
        elif i == 2:
            ones.append(0)
            twos.append(1)
        else:
            ones.append(0)
            twos.append(0)
        
    return np.array(ones, dtype=float), np.array(twos, dtype=float)

In [6]:
# -*- coding: utf-8 -*-
import torch

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

top = 2

# Use the nn package to define our model and loss function.
p1 = torch.nn.Sequential(
    torch.nn.Linear(top**0, top**1),
    torch.nn.ReLU(),
    torch.nn.Linear(top**1, top**2),
    torch.nn.ReLU(),
    torch.nn.Linear(top**2, top**3),
    torch.nn.ReLU(),
    torch.nn.Linear(top**3, top**4),
    torch.nn.ReLU(),
    torch.nn.Linear(top**4, top**5),
    torch.nn.ReLU(),
    torch.nn.Linear(top**5, top**6),
    torch.nn.ReLU(),
    torch.nn.Linear(top**6, top**7),
    torch.nn.ReLU(),
    torch.nn.Linear(top**7, top**8),
    torch.nn.ReLU(),
    torch.nn.Linear(top**8, top**9),
    torch.nn.ReLU(),
    torch.nn.Linear(top**9, top**10),
    torch.nn.ReLU(),
    torch.nn.Linear(top**10, top**11),
    torch.nn.ReLU(),
    torch.nn.Linear(top**11, top**12),
    torch.nn.Sigmoid(),
)


p2 = torch.nn.Sequential(
    torch.nn.Linear(top**0, top**1),
    torch.nn.ReLU(),
    torch.nn.Linear(top**1, top**2),
    torch.nn.ReLU(),
    torch.nn.Linear(top**2, top**3),
    torch.nn.ReLU(),
    torch.nn.Linear(top**3, top**4),
    torch.nn.ReLU(),
    torch.nn.Linear(top**4, top**5),
    torch.nn.ReLU(),
    torch.nn.Linear(top**5, top**6),
    torch.nn.ReLU(),
    torch.nn.Linear(top**6, top**7),
    torch.nn.ReLU(),
    torch.nn.Linear(top**7, top**8),
    torch.nn.ReLU(),
    torch.nn.Linear(top**8, top**9),
    torch.nn.ReLU(),
    torch.nn.Linear(top**9, top**10),
    torch.nn.ReLU(),
    torch.nn.Linear(top**10, top**11),
    torch.nn.ReLU(),
    torch.nn.Linear(top**11, top**12),
    torch.nn.Sigmoid(),
)


loss_fn = torch.nn.MSELoss()

# Use the optim package to define an Optimizer that will update the weights of
# the model for us. Here we will use Adam; the optim package contains many other
# optimization algoriths. The first argument to the Adam constructor tells the
# optimizer which Tensors it should update.
learning_rate = 0.001
optimizer_p1 = torch.optim.Adam(p1.parameters(), lr=learning_rate)
optimizer_p2 = torch.optim.Adam(p2.parameters(), lr=learning_rate)
true_board = np.ones((512*512), dtype=float)

for t in range(100000):
    # Forward pass: compute predicted y by passing x to the model.
    
    start_p1 = torch.rand(1)
    start_p2 = torch.rand(1)
    
    p1_board = p1(start_p1)
    p2_board = p2(start_p2)

    with torch.no_grad():
        p1_clone = p1_board.clone()
        p2_clone = p2_board.clone()
        
        p1_board_org = p1_clone.detach().numpy()
        p2_board_org = p2_clone.detach().numpy()
        
        p1_board_np = p1_clone.detach().numpy()
        p2_board_np = p2_clone.detach().numpy()
        
        for i in range(len(p1_board_org)):
            p1_board_np[i] = int(p1_board_np[i] + 0.5)
        for i in range(len(p2_board_np)):
            p2_board_np[i] = int(p2_board_np[i] + 0.5)
        
        p1_board_np = np.array(p1_board_np, dtype=np.int)
        p2_board_np = np.array(p2_board_np, dtype=np.int)
        p1_board_np = np.reshape(p1_board_np, (64, 64))
        p2_board_np = np.reshape(p2_board_np, (64, 64))
        
        game.add_players(p1_board_np, p2_board_np)
        game.launch(100)
        game_ints = game.gol.grid()
        game_ints = np.array(game_ints).flatten()

        p1_result, p2_result = return_result(game_ints)
        
        total_loss_p1 = ((p1_result - true_board) ** 2).mean(axis=None)
        total_loss_p2 = ((p2_result - true_board) ** 2).mean(axis=None)
        
        #print(total_loss_p1, total_loss_p2)
        
        p1_choice = [total_loss_p1, total_loss_p1]
        p2_choice = [total_loss_p2, total_loss_p2]
        
        p1_loss_board = np.random.choice(p1_choice, (64*64), p=[0.5, 0.5])
        p2_loss_board = np.random.choice(p2_choice, (64*64), p=[0.5, 0.5])
        
        y_p1 = torch.tensor((p1_loss_board + p1_board_org), dtype=torch.float)
        y_p2 = torch.tensor((p2_loss_board + p2_board_org), dtype=torch.float)
        
        
    
    loss_p1 = loss_fn(p1_board, y_p1)
    loss_p2 = loss_fn(p2_board, y_p2)
    
    # Train player 2
    optimizer_p1.zero_grad()
    optimizer_p2.zero_grad()
    
    loss_p1.backward()
    loss_p2.backward()
    
    optimizer_p1.step()
    optimizer_p2.step()
    
    print(loss_p1, loss_p2)
    if t % 100 == 0:
        turn_program_into_file(p1_board_np, save_dir + "p1-cycle-" + str(t) + ".rle", "RKH", "EW", "")
        turn_program_into_file(p2_board_np, save_dir + "p2-cycle-" + str(t) + ".rle", "RKH", "EW", "")

tensor(1.2299) tensor(1.2260)


FileNotFoundError: [Errno 2] No such file or directory: './output/KOH/1531323694/p1-cycle-0.rle'